In [16]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [17]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [18]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 41.0138,
    "longitude": 28.9497,
    "start_date": "2005-01-01",
    "end_date": "2021-04-07",
    "daily": [
        "weather_code",
        "temperature_2m_max",
        "temperature_2m_min",
        "temperature_2m_mean",
        "apparent_temperature_max",
        "apparent_temperature_min",
        "apparent_temperature_mean",
        "sunrise",
        "sunset",
        "daylight_duration",
        "sunshine_duration",
        "precipitation_sum",
        "rain_sum",
        "snowfall_sum",
        "precipitation_hours",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "wind_direction_10m_dominant",
        "shortwave_radiation_sum",
        "et0_fao_evapotranspiration",
    ],
    "timezone": "Europe/Moscow",
}
responses = openmeteo.weather_api(url, params=params)

In [19]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 41.01932907104492°N 28.887304306030273°E
Elevation 32.0 m asl
Timezone b'Europe/Moscow' b'GMT+3'
Timezone difference to GMT+0 10800 s


In [20]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()

daily_data = {
    "date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
    )
}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data=daily_data)

In [22]:
daily_dataframe

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,...,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2004-12-31 21:00:00+00:00,53.0,10.563000,7.3630,8.990084,9.071384,4.146955,6.033844,0,0,...,25582.630859,3.500000,3.500000,0.0,10.0,23.507751,37.439999,31.536366,5.190000,0.814421
1,2005-01-01 21:00:00+00:00,51.0,7.813000,4.7130,6.660917,4.816814,2.181339,3.769840,0,0,...,22339.408203,2.800000,2.800000,0.0,17.0,13.276144,25.919998,349.913513,3.920000,0.644396
2,2005-01-02 21:00:00+00:00,51.0,8.613000,2.2630,6.031750,5.408062,-1.033089,3.009735,0,0,...,27602.402344,0.100000,0.100000,0.0,1.0,13.708390,26.280001,260.346802,6.450000,0.863363
3,2005-01-03 21:00:00+00:00,53.0,6.763000,2.4630,5.388000,3.987309,-1.269303,1.578146,0,0,...,4671.306152,6.700000,6.700000,0.0,16.0,21.252199,31.680000,4.656595,2.600000,0.504352
4,2005-01-04 21:00:00+00:00,51.0,9.662999,4.3630,7.417167,6.653760,-0.225449,3.145541,0,0,...,25796.974609,0.100000,0.100000,0.0,1.0,25.499081,38.160000,248.799698,6.450000,1.421847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5936,2021-04-02 21:00:00+00:00,53.0,17.495501,9.7455,12.814251,16.171028,6.614742,10.190885,0,0,...,40197.460938,2.200000,2.200000,0.0,4.0,22.171440,37.439999,226.313049,19.889999,3.238446
5937,2021-04-03 21:00:00+00:00,53.0,14.895500,8.4955,11.751751,12.302643,4.086198,7.981377,0,0,...,29840.740234,2.500000,2.500000,0.0,9.0,29.363731,50.399998,267.299408,12.390000,2.114918
5938,2021-04-04 21:00:00+00:00,63.0,8.745500,5.1455,7.722583,6.753678,2.363430,4.550266,0,0,...,6535.202148,16.700001,16.700001,0.0,15.0,27.645092,80.279999,18.435013,6.720000,1.020160
5939,2021-04-05 21:00:00+00:00,53.0,18.145500,5.7955,11.935081,18.918333,3.684489,10.816017,0,0,...,35123.308594,2.400000,2.400000,0.0,8.0,15.328561,30.960001,71.375259,17.420000,2.648225


In [23]:
daily_dataframe.to_csv("../dataset/external/weather.csv", index=False)